# Lab 03 (Intermediate) — Python for AI

Python 기초를 이미 알고 있다는 전제 하에, 데이터 분석·머신러닝에서 실제로 자주 쓰이는 패턴을 집중적으로 학습합니다.

**학습 목표:**
1. Python 중급 패턴 — comprehension, OOP, 예외처리, 타입힌트
2. NumPy 심화 — 선형대수, 벡터화 연산 vs 루프 성능 비교
3. Pandas 심화 — 실제 데이터 클리닝, merge, pivot, apply
4. EDA 실습 — Titanic 데이터셋 탐색적 데이터 분석

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False
sns.set_theme(style="whitegrid")

---
## Part 1. Python 중급 패턴

### 1-1. Comprehension

In [ ]:
# List comprehension — 가장 흔하게 사용
squares = [x**2 for x in range(10) if x % 2 == 0]
print("짝수 제곱:", squares)

# Dict comprehension
word_len = {w: len(w) for w in ["apple", "banana", "cherry"]}
print("단어 길이:", word_len)

# Set comprehension — 중복 제거
labels = ["cat", "dog", "cat", "bird", "dog"]
unique_labels = {lbl.upper() for lbl in labels}
print("고유 레이블:", unique_labels)

# 중첩 comprehension — 2D → 1D flatten
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [v for row in matrix for v in row]
print("flatten:", flat)

### 1-2. Lambda · map · filter · zip

In [ ]:
# lambda — 이름 없는 짧은 함수
normalize = lambda x, lo, hi: (x - lo) / (hi - lo)
print(normalize(75, 0, 100))

# map — 각 요소에 함수 적용
raw = [" Alice ", " Bob ", " Charlie "]
cleaned = list(map(str.strip, raw))
print(cleaned)

# filter — 조건을 만족하는 요소만
scores = [55, 72, 88, 91, 43, 66]
passed = list(filter(lambda s: s >= 60, scores))
print("통과:", passed)

# zip — 두 시퀀스를 묶어 쌍으로
names  = ["Alice", "Bob", "Charlie"]
scores = [88, 75, 92]
paired = list(zip(names, scores))
print("쌍:", paired)

# zip으로 딕셔너리 생성
record = dict(zip(names, scores))
print("딕셔너리:", record)

### 1-3. 예외 처리 (Exception Handling)

In [ ]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        print("오류: 0으로 나눌 수 없습니다.")
        return None
    except TypeError as e:
        print(f"타입 오류: {e}")
        return None

print(safe_divide(10, 2))
print(safe_divide(10, 0))
print(safe_divide(10, "a"))

In [ ]:
# 데이터 파이프라인에서 예외 처리 활용
raw_data = ["3.14", "2.71", "N/A", "1.41", ""]

def to_float(s):
    try:
        return float(s)
    except (ValueError, TypeError):
        return np.nan

cleaned = [to_float(v) for v in raw_data]
print(cleaned)

### 1-4. 클래스 (OOP 기초)

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Student:
    name:   str
    scores: List[float] = field(default_factory=list)

    @property
    def average(self) -> float:
        return sum(self.scores) / len(self.scores) if self.scores else 0.0

    @property
    def grade(self) -> str:
        avg = self.average
        if avg >= 90: return "A"
        if avg >= 80: return "B"
        if avg >= 70: return "C"
        return "F"

    def __repr__(self):
        return f"{self.name} | 평균 {self.average:.1f} | 학점 {self.grade}"


students = [
    Student("Alice",   [88, 92, 85]),
    Student("Bob",     [72, 68, 75]),
    Student("Charlie", [95, 98, 91]),
]

for s in students:
    print(s)

# 정렬
top = sorted(students, key=lambda s: s.average, reverse=True)
print("\n순위:", [s.name for s in top])

### 1-5. 타입 힌트 (Type Hints)

In [ ]:
from typing import Optional, Tuple

def train_test_split_idx(
    n: int,
    test_ratio: float = 0.2,
    seed: Optional[int] = None
) -> Tuple[List[int], List[int]]:
    """인덱스를 train/test 로 분할."""
    rng = np.random.default_rng(seed)
    idx = rng.permutation(n).tolist()
    split = int(n * (1 - test_ratio))
    return idx[:split], idx[split:]

train_idx, test_idx = train_test_split_idx(20, seed=42)
print("train:", train_idx)
print("test: ", test_idx)

> **Exercise 1.** 아래 함수를 완성하세요.  
> `normalize_list(data: List[float]) -> List[float]`  
> — 리스트를 Min-Max 정규화([0, 1] 범위)하여 반환합니다. 예외 처리(길이 0, 모두 같은 값)도 포함하세요.

In [ ]:
def normalize_list(data: List[float]) -> List[float]:
    # Your code here
    pass

print(normalize_list([10, 20, 30, 40, 50]))  # [0.0, 0.25, 0.5, 0.75, 1.0]
print(normalize_list([5, 5, 5]))             # 모두 같은 값 → 처리 필요

---
## Part 2. NumPy 심화

### 2-1. 브로드캐스팅 규칙

In [ ]:
# 규칙: 차원이 맞지 않으면 크기가 1인 차원을 확장
A = np.array([[1], [2], [3]])   # shape (3, 1)
B = np.array([10, 20, 30])      # shape (3,) → (1, 3)

print("A shape:", A.shape)
print("B shape:", B.shape)
print("A + B:\n", A + B)        # 결과 shape (3, 3)

In [ ]:
# 실용 예 — 데이터 표준화 (Z-score)
X = np.array([[2, 4, 6],
              [1, 3, 5],
              [3, 5, 7]], dtype=float)

mean = X.mean(axis=0)  # 열(feature)별 평균, shape (3,)
std  = X.std(axis=0)   # 열별 표준편차

Z = (X - mean) / std   # 브로드캐스팅으로 한 번에 처리
print("표준화:\n", Z.round(3))
print("열별 평균:", Z.mean(axis=0).round(10))  # ≈ 0
print("열별 표준편차:", Z.std(axis=0).round(10))   # ≈ 1

### 2-2. 벡터화 vs 루프 — 성능 비교

In [ ]:
import time

N = 1_000_000
data = np.random.rand(N)

# 방법 1: Python 루프
start = time.perf_counter()
total = 0
for v in data:
    total += v
t_loop = time.perf_counter() - start

# 방법 2: NumPy 벡터화
start = time.perf_counter()
total_np = data.sum()
t_np = time.perf_counter() - start

print(f"Python 루프 : {t_loop*1000:.1f} ms")
print(f"NumPy 벡터화: {t_np*1000:.3f} ms")
print(f"속도 향상   : {t_loop / t_np:.0f}x")

### 2-3. 선형대수 (np.linalg)

In [ ]:
# 행렬 곱
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
print("A @ B:\n", A @ B)              # @ 연산자 = matmul

# 역행렬
A_inv = np.linalg.inv(A)
print("\nA 역행렬:\n", A_inv.round(3))
print("A @ A_inv (≈ I):\n", (A @ A_inv).round(10))

# 고유값 분해 (PCA 등에서 핵심)
eigenvalues, eigenvectors = np.linalg.eig(A)
print("\n고유값:", eigenvalues.round(3))
print("고유벡터:\n", eigenvectors.round(3))

In [ ]:
# 최소제곱법으로 선형 회귀 계수 구하기
# y = w0 + w1*x  →  X @ w = y
rng = np.random.default_rng(0)
x = np.linspace(0, 10, 50)
y = 2.5 * x + 1.0 + rng.normal(0, 1, 50)  # 실제 w0=1, w1=2.5

X = np.column_stack([np.ones(len(x)), x])  # 편향 항 추가
w, residuals, rank, sv = np.linalg.lstsq(X, y, rcond=None)

print(f"추정 w0 (절편): {w[0]:.3f}  (실제: 1.0)")
print(f"추정 w1 (기울기): {w[1]:.3f}  (실제: 2.5)")

### 2-4. 유용한 함수들

In [ ]:
a = np.array([3, 1, 4, 1, 5, 9, 2, 6, 5, 3])

# np.where — 조건에 따라 값 선택
clipped = np.where(a > 5, 5, a)   # 5 초과면 5로 대체
print("클리핑:", clipped)

# np.unique — 고유값과 빈도
vals, counts = np.unique(a, return_counts=True)
print("고유값:", vals)
print("빈도:  ", counts)

# np.argsort — 정렬된 인덱스
idx = np.argsort(a)[::-1]   # 내림차순
print("내림차순 인덱스:", idx)
print("정렬 결과:      ", a[idx])

> **Exercise 2.** `(100, 5)` 크기의 랜덤 행렬 `X`를 생성하고,  
> 각 열(feature)을 Z-score 정규화한 뒤 정규화 후 각 열의 평균과 표준편차를 확인하세요.

In [ ]:
# Your code here


---
## Part 3. Pandas 심화

### 3-1. Titanic 데이터 로딩

In [ ]:
titanic = sns.load_dataset("titanic")
print(titanic.shape)
titanic.head()

### 3-2. 데이터 클리닝 파이프라인

In [ ]:
# 결측치 현황
missing = titanic.isnull().sum()
missing_pct = (missing / len(titanic) * 100).round(1)
pd.DataFrame({"결측 수": missing, "결측 비율(%)": missing_pct}).query("`결측 수` > 0")

In [ ]:
df = titanic.copy()

# age: 중앙값으로 대체
df["age"] = df["age"].fillna(df["age"].median())

# embarked: 최빈값으로 대체
df["embarked"] = df["embarked"].fillna(df["embarked"].mode()[0])

# deck: 결측 비율이 너무 높아 열 제거
df = df.drop(columns=["deck"])

# 중복 행 확인 및 제거
print("중복 행:", df.duplicated().sum())

print("\n클리닝 후 결측치:")
print(df.isnull().sum()[df.isnull().sum() > 0])

### 3-3. 특성 엔지니어링

In [ ]:
# 가족 크기
df["family_size"] = df["sibsp"] + df["parch"] + 1
df["is_alone"] = (df["family_size"] == 1).astype(int)

# 나이 그룹
df["age_group"] = pd.cut(
    df["age"],
    bins=[0, 12, 18, 60, 100],
    labels=["child", "teen", "adult", "senior"]
)

# 운임 로그 변환 (왜도 감소)
df["log_fare"] = np.log1p(df["fare"])

df[["family_size", "is_alone", "age_group", "log_fare"]].head()

### 3-4. groupby 심화

In [ ]:
# 다중 집계
survival_stats = df.groupby(["pclass", "sex"])["survived"].agg(
    count="count",
    survivors="sum",
    survival_rate="mean"
).round(3)

survival_stats

In [ ]:
# transform — 그룹 통계를 원본 행에 붙이기
df["class_avg_fare"] = df.groupby("pclass")["fare"].transform("mean")
df["fare_vs_class_avg"] = df["fare"] - df["class_avg_fare"]

df[["pclass", "fare", "class_avg_fare", "fare_vs_class_avg"]].head(8)

### 3-5. pivot_table

In [ ]:
pivot = pd.pivot_table(
    df,
    values="survived",
    index="pclass",
    columns="sex",
    aggfunc="mean"
).round(3)

pivot

### 3-6. apply + 커스텀 함수

In [ ]:
def fare_tier(row):
    """객실 등급과 운임을 기반으로 티어 분류"""
    if row["pclass"] == 1 and row["fare"] > 50:
        return "premium"
    elif row["pclass"] <= 2:
        return "standard"
    else:
        return "economy"

df["tier"] = df.apply(fare_tier, axis=1)
df[["pclass", "fare", "tier", "survived"]].head(10)

> **Exercise 3.** 나이 그룹(`age_group`)별 생존율을 계산하고,  
> 어느 그룹의 생존율이 가장 높은지 확인하세요.

In [ ]:
# Your code here


---
## Part 4. EDA — Titanic 탐색적 데이터 분석

### 4-1. 생존율 개요

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 전체 생존 비율
df["survived"].value_counts().rename({0: "사망", 1: "생존"}).plot(
    kind="bar", ax=axes[0], color=["tomato", "steelblue"], edgecolor="white"
)
axes[0].set_title("생존 vs 사망")
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# 객실 등급별 생존율
df.groupby("pclass")["survived"].mean().plot(
    kind="bar", ax=axes[1], color="steelblue", edgecolor="white"
)
axes[1].set_title("객실 등급별 생존율")
axes[1].set_xlabel("객실 등급")
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

# 성별 생존율
df.groupby("sex")["survived"].mean().plot(
    kind="bar", ax=axes[2], color=["steelblue", "tomato"], edgecolor="white"
)
axes[2].set_title("성별 생존율")
axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=0)

for ax in axes:
    ax.set_ylim(0, 1)
    ax.set_ylabel("생존율")

plt.suptitle("Titanic 생존율 분석", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 4-2. 나이 분포

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 나이 히스토그램 (생존 여부 구분)
for survived, label, color in [(0, "사망", "tomato"), (1, "생존", "steelblue")]:
    axes[0].hist(
        df[df["survived"] == survived]["age"],
        bins=30, alpha=0.6, label=label, color=color, edgecolor="white"
    )
axes[0].set_title("나이 분포 (생존 여부)")
axes[0].set_xlabel("나이")
axes[0].set_ylabel("인원 수")
axes[0].legend()

# KDE plot
sns.kdeplot(data=df, x="age", hue="survived", ax=axes[1],
            palette={0: "tomato", 1: "steelblue"}, fill=True, alpha=0.4)
axes[1].set_title("나이 밀도 분포")
axes[1].set_xlabel("나이")

plt.tight_layout()
plt.show()

### 4-3. 운임 분포와 이상치

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 원본 운임 박스플롯
sns.boxplot(data=df, x="pclass", y="fare", ax=axes[0],
            hue="pclass", palette="Set2", legend=False)
axes[0].set_title("운임 분포 (원본)")
axes[0].set_xlabel("객실 등급")

# 로그 변환 후
sns.boxplot(data=df, x="pclass", y="log_fare", ax=axes[1],
            hue="pclass", palette="Set2", legend=False)
axes[1].set_title("운임 분포 (log 변환 후)")
axes[1].set_xlabel("객실 등급")

plt.tight_layout()
plt.show()

### 4-4. 히트맵 — 성별·등급별 생존율

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(
    pivot,
    annot=True, fmt=".2f", cmap="RdYlGn",
    vmin=0, vmax=1,
    linewidths=0.5
)
plt.title("성별 × 객실 등급 생존율")
plt.tight_layout()
plt.show()

### 4-5. Pairplot — 수치형 변수 간 관계

In [ ]:
cols = ["age", "fare", "family_size", "survived"]
g = sns.pairplot(
    df[cols].dropna(),
    hue="survived",
    palette={0: "tomato", 1: "steelblue"},
    plot_kws={"alpha": 0.5},
    diag_kind="kde"
)
g.figure.suptitle("Titanic Pairplot", y=1.02)
plt.show()

### 4-6. 상관관계 분석

In [ ]:
num_cols = ["survived", "pclass", "age", "fare", "family_size", "is_alone", "log_fare"]
corr = df[num_cols].corr()

# survived와의 상관계수만 추출
print("생존과의 상관계수:")
print(corr["survived"].drop("survived").sort_values(ascending=False).round(3))

In [ ]:
plt.figure(figsize=(7, 6))
mask = np.triu(np.ones_like(corr, dtype=bool))   # 상삼각 마스크
sns.heatmap(
    corr, mask=mask,
    annot=True, fmt=".2f", cmap="coolwarm",
    vmin=-1, vmax=1, linewidths=0.5
)
plt.title("수치형 변수 상관관계")
plt.tight_layout()
plt.show()

> **Exercise 4.** `tier` 열(premium / standard / economy)별로  
> 생존율을 계산하고, 막대 그래프로 시각화하세요.

In [ ]:
# Your code here


---
## Summary

| 파트 | 핵심 개념 |
|---|---|
| Python 중급 | comprehension, lambda/map/filter/zip, 예외처리, dataclass, 타입힌트 |
| NumPy 심화 | 브로드캐스팅, 벡터화 성능, 선형대수(linalg), 최소제곱법 |
| Pandas 심화 | 데이터 클리닝, 특성 엔지니어링, groupby+transform, pivot_table, apply |
| EDA | 분포 시각화, 이상치 탐지, pairplot, 상관관계 히트맵 |

**EDA 주요 발견 (Titanic):**
- 여성의 생존율이 남성보다 훨씬 높음 ("women and children first")
- 1등 객실 생존율 >> 3등 객실
- 운임(`fare`)과 생존 간 양의 상관관계
- 혼자 탑승한 승객(`is_alone`)의 생존율이 상대적으로 낮음